## Bestäm vilka kryptovalutor som ska användas i yFinance
Tar fram alla kryptovalutor som finns på Binance och deras historiska Close-priser  
Därefter selekterar vi ut och behåller de kolumner som inte har för många NaN's  

Slutligen Testar vi vilka av dessa som kan användas i yfinance  
Spara undan dessa ticker-namn i en fil  

In [83]:
# Importerar bibliotek

import pandas as pd 
import os 

from binance.client import Client
import yfinance as yf 
import datetime

In [84]:
"""
    Funktionen hämta_från_scalers() hämtar ticker-namn från en specifik mapp som heter "scalers". 
    Dessa ticker-namn representerar olika kryptovalutor. 
    Funktionen returnerar en lista med dessa namn.
"""
def hämta_från_scalers():
    # Detta fungerar enbart om vi har sparat StdScaler för Binance-tickers i mappen "scalers"
    folder_path = "scalers/"
    file_names = os.listdir(folder_path)

    binance_names = []
    for file_name in file_names:
        if file_name.endswith("_scaler.joblib"):
            crypto_name = file_name[:-14]  # 14 är längden på "_scaler.joblib"
            binance_names.append(crypto_name) # type: ignore

    print(len(binance_names),binance_names)

    return binance_names


In [85]:

def hämta_från_Binance():
    """
    Funktionen hämta_från_Binance() samlar in historisk prisinformation 
    för olika kryptovalutor från Binance. 
    Den hämtar informationen via Binance's API och returnerar en pandas DataFrame 
    som innehåller all denna information.
    Denna kör vi om hämta_från_scalers() inte är aktuell
    """
    
    MAX_MONTHS = 24
    api_key = '2jxiCQ8OIWmU4PZH4xfwKEY9KYerDkSWzwNCqoaMzj41eJgWBsSqA3VYqkt2wmdX'
    api_secret = 'YY1Qj1t0JZrE4tQdaBBxT8iwl2tbFalWp1FHjyZ9selBb6OnQ0Oj8aVdiXO7YLMz'

    client = Client(api_key, api_secret)

    # Hämta handelspar
    symbols = client.get_all_tickers()
    symbols = [
        symbol for symbol in symbols if symbol['symbol'].endswith('USDT')]

    # Sätt upp en tom lista för att lagra close-priser
    close_prices = {}

    # Ange den tidsram du vill ha för historiska data
    interval = Client.KLINE_INTERVAL_1DAY
    start_time = f"{MAX_MONTHS} month ago UTC"

    # Hämta close-priser för alla kryptovalutor
    dates = None

    for idx, symbol in enumerate(symbols):
        try:
            klines = client.get_historical_klines(
                symbol['symbol'], interval, start_time)
            if dates is None:
                # Extrahera och konvertera tidsstämplar till datum
                dates = [datetime.datetime.fromtimestamp(
                    int(kline[0]) / 1000).strftime('%Y-%m-%d') for kline in klines]
            close_prices[symbol['symbol']] = [
                float(kline[4]) for kline in klines]
        except Exception as e:
            print(f"Kunde inte hämta data för {symbol['symbol']}: {e}")

        # Update the progress bar
        
        if idx+1 == len(symbols):
            progress_text = f"Done! last symbol . . . . . . . . {symbol['symbol']} fetched."
        elif (idx+1) % 20 == 0:
            progress_text = f"This will take several minutes. Symbol number {idx+1} of {len(symbols)}: . . . . . . . . {symbol['symbol']}"
            print(progress_text)

    # Konvertera close_prices-dikten till en pandas DataFrame
    df = pd.DataFrame.from_dict(close_prices, orient='index').transpose()
    # Lägg till datum som index för DataFrame
    # type: ignore
    df.index = pd.to_datetime(dates)  # type: ignore
    # print(df.head())
    print(len(df.columns),  df.columns.tolist())
    return df


In [ ]:
# get all Binance tickers Close prices

if False:  # om scalers innehåller Binance och är aktuella 
    binance_names = hämta_från_scalers()
else:
    df_binance = hämta_från_Binance()
    binance_names = df_binance.columns.tolist()
    
# Omvandlar koden ticker-namnen från Binance-format till yfinance-format.  
yf_names = [name.replace("USDT", "-USD") for name in binance_names]
print(len(yf_names),yf_names)

### Kolla vilka som finns i yfinace

In [ ]:
# Funktionen get_filtered_names() tar en lista med ticker-namn och returnerar en ny lista 
# som endast innehåller de ticker-namn som är giltiga i yfinance.

def get_filtered_names(yf_names):
    # Hämta giltiga tickers från yfinance
    valid_tickers = set()

    for name in yf_names:
        ticker = yf.Ticker(name)
        if ticker.history(period="5d").empty == False:  # Kollar om ticker har data för att avgöra om den är giltig eller inte
            valid_tickers.add(name)

    filtered_names = list(set(yf_names).intersection(valid_tickers))

    print(len(filtered_names),filtered_names)
    return filtered_names
filtered_names = get_filtered_names(yf_names)

In [101]:
len(filtered_names)

365

In [102]:
# get the close-prices for the tickers in filtered_names. Use yfinance. 
time_period = '24mo'  # 24 months back
interval = '1d'  # daily data
def get_close_prices(filtered_names):    
    data = yf.download(filtered_names, interval='1d', period=time_period, group_by='ticker', auto_adjust=True)
    df = pd.DataFrame(data.xs('Close', level=1, axis=1))
    return df

df = get_close_prices(filtered_names)


[*********************100%***********************]  365 of 365 completed
730 days 00:00:00 från 2021-05-09 00:00:00 till 2023-05-09 00:00:00


In [103]:

print((df.index[-1].date() - df.index[0].date()).days, 'dagar: ', df.index[0].date(),'-',df.index[-1].date())

730 dagar:  2021-05-09 - 2023-05-09


In [104]:

def preprocessing_currency(df_):
    # kontrollera att index är dateindex
    if type(df_.index[0]) != pd.Timestamp:
        # break with error
        print('index är inte dateindex')
        return None
    
    df = df_.copy()
    df = df.drop_duplicates()
    print(len(df.columns), 'kolumner totalt')
    s=int(0.9*len(df))
    # remove all columns where there are more than s nan values
    df = df.dropna(axis=1, thresh=s)
    print(len(df.columns), f'kolumner med minst {s} rader utan nan efter dropna')
    print(f'{df.isna().sum().sum()} rader med någon nan \n{len(df)-df.isna().any().sum()} rader utan nan')
    # interpolate missing values
    df.interpolate(method='linear', inplace=True)
    print(f'Efter interpollate: {df.isna().any().sum()} rader med någon nan \n{len(df)-df.isna().any().sum()} rader utan nan')
    print(df.shape)
    
    return df

df = preprocessing_currency(df)

365 kolumner totalt
321 kolumner med minst 657 rader utan nan efter dropna
650 rader med någon nan 
508 rader utan nan
Efter interpollate: 11 rader med någon nan 
720 rader utan nan
(731, 321)


### Spara alla godkända tickers i yf till en fil

In [108]:
# spara df.columns i en lista till en fil
filnamn='yf_tickers.txt'
with open( filnamn, 'w') as f:
    for item in df.columns.tolist():
        f.write("%s\n" % item)
print(f'Lista med {len(df.columns)} godkända tickers sparad i {filnamn}')


Lista med 321 godkända tickers sparad iyf_tickers.txt
